# Fetch Rewards Coding Exercise - Analytics Engineer

## 1. Review Existing Unstructured Data and Diagram a New Structured Relational Data Model

### 1.1. Relational Data Model

<a id="section_1_data_model"></a>

Diagram: [section_1_data_model.jpg](section_1_data_model.jpg)
![](section_1_data_model.jpg)

### 1.2. Design decisions for the relation diagram

For preliminary analysis, I started with identifying the following things for each table:
- Data type of each column
- Duplicate values in a column
- If the data in a column contains null values
- If the data is a column represents a JSON object or structure that needs to be extracted out in a separate table

In [1]:
import os
import pandas as pd

# constants for path to raw data files
USERS_JSON_PATH = os.path.join("data", "users.json")
BRANDS_JSON_PATH = os.path.join("data", "brands.json")
RECEIPTS_JSON_PATH = os.path.join("data", "receipts.json")

In [2]:
# Load user data
users = pd.read_json(USERS_JSON_PATH,lines=True)
users.rename(columns = {"_id":"id"}, inplace = True)
users.head(2)

,id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [3]:
# Load brand data
brands = pd.read_json(BRANDS_JSON_PATH,lines=True)
brands.rename(columns = {"_id":"id"}, inplace = True)
brands.head(2)

,id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS


In [4]:
# Load receipt data
receipts = pd.read_json(RECEIPTS_JSON_PATH,lines=True)
receipts.rename(columns = {"_id":"id"}, inplace = True)
receipts.head(2)

,id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052


The first thing that I notice is the presence of `$oid`, `$date` and `$ref` which suggests that the data is likely associated with MongoDB. The `id` for each row can be extracted from `$oid` and the date values can be parsed from `$date`.

In [5]:
# Function to find value by key in dict
def get_val(dict_x, key):
    return dict_x[key] if isinstance(dict_x, dict) and key in dict_x else None

# Transform id and datetime fields
users["id"] = users["id"].apply(lambda x: x["$oid"])
users["createdDate"] = pd.to_datetime(users["createdDate"].apply(lambda x: get_val(x, "$date")), unit="ms")
users["lastLogin"] = pd.to_datetime(users["lastLogin"].apply(lambda x: get_val(x, "$date")), unit="ms")
users.head(2)

,id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI


In [6]:
# Checking for null values
users.isnull().sum()

id               0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [7]:
# Checking different roles to decide if a separate table is needed
users["role"].value_counts()

consumer       413
fetch-staff     82
Name: role, dtype: int64

In [8]:
# Checking different signUpSource values to decide if a separate table is needed
users["signUpSource"].value_counts()

Email     443
Google      4
Name: signUpSource, dtype: int64

In [9]:
# Checking different state values present in the date
users["state"].value_counts()

WI    396
NH     20
AL     12
OH      5
IL      3
KY      1
CO      1
SC      1
Name: state, dtype: int64

In [10]:
# Checking if id column is unique in raw data
users["id"].nunique() == users["id"].count()

False

The `id` column has duplicate values which is a data quality issue.

#### 1.2.1. Finalizing `user` table

The user data does not contain any nested objects, hence I decided to have a single table as displayed in [data model diagram](#section_1_data_model).

**Future scope**: Further normalization can be performed to extract the columns `role`, `signUpSource` and `state` as tables, but I decided to keep the design simple since it was mentioned that the potential target for this data will be a data warehouse. Extracting these columns into table may result in extra `JOIN` operations while retrieving data. Also, since it a test, I thought its better to keep the design simple.

In [11]:
# Transform id
brands["id"] = brands["id"].apply(lambda x: x["$oid"])
brands.head(2)

,id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS


In [12]:
# Checking for nullable columns
brands.isnull().sum()

id                0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
dtype: int64

In [13]:
# Checking different roles to decide if a separate table is needed
brands["category"].value_counts()

Baking                         369
Beer Wine Spirits               90
Snacks                          75
Candy & Sweets                  71
Beverages                       63
Magazines                       44
Health & Wellness               44
Breakfast & Cereal              40
Grocery                         39
Dairy                           33
Condiments & Sauces             27
Frozen                          24
Personal Care                   20
Baby                            18
Canned Goods & Soups            12
Beauty                           9
Cleaning & Home Improvement      6
Deli                             6
Beauty & Personal Care           6
Household                        5
Bread & Bakery                   5
Dairy & Refrigerated             5
Outdoor                          1
Name: category, dtype: int64

In [14]:
# Checking different roles to decide if a separate table is needed
brands["categoryCode"].value_counts()

BAKING                           359
CANDY_AND_SWEETS                  71
BEER_WINE_SPIRITS                 31
HEALTHY_AND_WELLNESS              14
GROCERY                           11
BABY                               7
CLEANING_AND_HOME_IMPROVEMENT      6
BREAD_AND_BAKERY                   5
DAIRY_AND_REFRIGERATED             5
PERSONAL_CARE                      4
BEVERAGES                          1
OUTDOOR                            1
MAGAZINES                          1
FROZEN                             1
Name: categoryCode, dtype: int64

In [15]:
# Checking if id column is unique in raw data
brands["id"].nunique() == brands["id"].count()

True

In [16]:
# Checking if barcode column is unique in raw data
brands["barcode"].nunique() == brands["barcode"].count()

False

The `barcode` column has duplicate values which can be considered a data quality issue since barcodes need to be unique.

Normalization can be performed to extract the `category` and `categoryCode` column into a separate table, but to keep the design simple and maintain scope of this assessment, I decided to not create another table for it.

The `cpg` column of `brand` contains an object with the `$ref` property which suggests that another document collection is referenced in brands, which existed in the raw dataset. The `$ref` field points to two types of collections, `COGS` and `CPGS`. Since this reference field doesn't have any other fields to identify their exact nature and whether they are related or not, I decided to create two tables `cogs` and `cpgs`, each of which is linked to `brands` table using `cogs_id` and `cpgs_id` respectiveyly. Unlike `category`, I created these separate tables for cpg just to respect the existence of the `$ref` relation.

#### 1.2.2. Finalizing `brand` table

The `brand` table contains two `$ref` objects, `COGS` and `CPGS` and hence I decided to extra them out as separate tables. `cogs` and `cpgs` only have `id` columns as primary key for now, but can be extended to have more columns in the future. The table structure for `brand`, `cogs` and `cpgs` table is displayed in [data model diagram](#section_1_data_model).

**Future scope**: Further normalization can be performed to extract the columns `category` with columns `id`, `name` and `categoryCode`, but I decided to keep the design simple since it was mentioned for the same reason as mentioned in future scope for `users` table.

In [17]:
# Transform id and datetime fields
receipts["id"] = receipts["id"].apply(lambda x: x["$oid"])
receipts["createDate"] = pd.to_datetime(receipts["createDate"].apply(lambda x: get_val(x, "$date")), unit="ms")
receipts["dateScanned"] = pd.to_datetime(receipts["dateScanned"].apply(lambda x: get_val(x, "$date")), unit="ms")
receipts["finishedDate"] = pd.to_datetime(receipts["finishedDate"].apply(lambda x: get_val(x, "$date")), unit="ms")
receipts["modifyDate"] = pd.to_datetime(receipts["modifyDate"].apply(lambda x: get_val(x, "$date")), unit="ms")
receipts["pointsAwardedDate"] = pd.to_datetime(receipts["pointsAwardedDate"].apply(lambda x: get_val(x, "$date")), unit="ms")
receipts["purchaseDate"] = pd.to_datetime(receipts["purchaseDate"].apply(lambda x: get_val(x, "$date")), unit="ms")
receipts.head(2)

,id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052


In [18]:
# Checking for nullable columns
receipts.isnull().sum()

id                           0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

In [19]:
# Check receipt_items in receipts
receipts["rewardsReceiptItemList"]

0       [{'barcode': '4011', 'description': 'ITEM NOT ...
1       [{'barcode': '4011', 'description': 'ITEM NOT ...
2       [{'needsFetchReview': False, 'partnerItemId': ...
3       [{'barcode': '4011', 'description': 'ITEM NOT ...
4       [{'barcode': '4011', 'description': 'ITEM NOT ...
                              ...                        
1114    [{'barcode': 'B076FJ92M4', 'description': 'mue...
1115                                                  NaN
1116                                                  NaN
1117    [{'barcode': 'B076FJ92M4', 'description': 'mue...
1118                                                  NaN
Name: rewardsReceiptItemList, Length: 1119, dtype: object

The `rewardsReceiptItemList` contains a list of JSON object which needs to be translated into a structure of its own. Each item in this list can be extracted into a new table called `receipt_item` with a foreign key `receipt_id` which references the `id` of the receipt in which this item was found. 

In [20]:
rows = []

# Iterate over receipts
for index, row in receipts.iterrows():
    receipt_id = row["id"]
    items = row["rewardsReceiptItemList"]
    # Continue only if we find a list of items
    if not isinstance(items, list):
        continue
    try:
      for item in items:
          # Create a row for each item in receipt and add receipt_id field for mapping
          new_row = {**item, "receipt_id": receipt_id}
          rows.append(new_row)
    except Exception as e:
       print(items)
       

receipt_items = pd.DataFrame(rows).sort_index(axis=1)
receipt_items.head(2)

,barcode,brandCode,competitiveProduct,competitorRewardsGroup,deleted,description,discountedItemPrice,finalPrice,itemNumber,itemPrice,...,quantityPurchased,receipt_id,rewardsGroup,rewardsProductPartnerId,targetPrice,userFlaggedBarcode,userFlaggedDescription,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity
0,4011,NaN,NaN,NaN,NaN,ITEM NOT FOUND,NaN,26.00,NaN,26.00,...,5.0,5ff1e1eb0a720f0523000575,NaN,NaN,NaN,4011,NaN,True,26.00,5.0
1,4011,NaN,NaN,NaN,NaN,ITEM NOT FOUND,NaN,1,NaN,1,...,1.0,5ff1e1bb0a720f052300056b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It can be observed that there are no nested objects in `receipt_items`, hence no further normalization is required and hence a single table `receipt_item` can be created for it.

It is crucial to establish a relation between `receipt_item` and `brand`. On first glance, `brandCode` seems to be potential candidate for this relation as even `brand` table contains `brandCode`. Let's analyze if `brandCode` can be used for this purpose.

In [21]:
# Checking for nullable columns
receipt_items.isnull().sum()

barcode                               3851
brandCode                             4341
competitiveProduct                    6296
competitorRewardsGroup                6666
deleted                               6932
description                            381
discountedItemPrice                   1172
finalPrice                             174
itemNumber                            6788
itemPrice                              174
metabriteCampaignId                   6078
needsFetchReview                      6128
needsFetchReviewReason                6722
originalFinalPrice                    6932
originalMetaBriteBarcode              6870
originalMetaBriteDescription          6931
originalMetaBriteItemPrice            6932
originalMetaBriteQuantityPurchased    6926
originalReceiptItemText               1181
partnerItemId                            0
pointsEarned                          6014
pointsNotAwardedReason                6601
pointsPayerId                         5674
preventTarg

In [22]:
# Check if all brandCodes of receipt_items are present in brandCodes of brands
receipt_items["brandCode"].isin(brands["brandCode"]).all()

False

Not all items in `receipt_items` contain `brandCode` and `brands` does not have all the `brandCode`s present in `receipt_items`. Hence `brandCode` is not a reliable column for establishing relation between these two tables. Another option was to use `barcode` which is also present in `brand` table, but by definition, a barcode should be unique for each item and hence I don't think it is the right field to join the two tables. I decided to add a new column in `receipt_items` called `brand_id` which will link the item to a brand. This column needs to be populated for each item but I think this is the right approach to form a relation between the two tables.

#### 1.2.3. Finalizing `receipt` and `receipt_item` table

The `receipt` table will not have `rewardsReceiptItemList` as it is extracted into a separate table called `receipt_item`. The `receipt_item` table has a field called `receipt_id` that references `id` of `receipt` to establish a one to many relation betweeen receipt and its items. The table structure for `receipt` and `receipt_item` is displayed in [data model diagram](#section_1_data_model).

## 2. Write queries that directly answer predetermined questions from a business stakeholder

### 2.1. What are the top 5 brands by receipts scanned for most recent month?

SQL file: [Top 5 brands by receipts scanned for most recent month](./section_2/1_top_5_brands_receipts_scanned_recent_month.sql)

```sql
WITH recent_month AS ( -- CTE to find the most recent month present in the receipt table
    SELECT DATE_TRUNC('month', MAX(date_scanned)) AS recent_month -- maximum value of the date_scanned column and truncating it to the beginning of the month
    FROM receipt
)
SELECT brand.name, COUNT(receipt_id) AS count
FROM receipt
JOIN receipt_item ON receipt.id = receipt_item.receipt_id -- join receipt_item with receipt with receipt_id
JOIN brand ON receipt_item.brand_id = brand.id -- join receipt_item with brand with brand_id
WHERE DATE_TRUNC('month', receipt.date_scanned) = (SELECT recent_month FROM recent_month) -- filter to include only those from the most recent month found in the recent_month CTE
GROUP BY brand.name
ORDER BY count DESC -- order based on receipt count
LIMIT 5; -- get top 5 results
```

### 2.2. How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

SQL file: [Comparison of ranking of top 5 brands by receipts scanned for the recent month to previous month](./section_2/2_top_5_brands_rank_comparison.sql)

```sql
WITH recent_months AS ( -- CTE to find the most recent two month present in the receipt table
    SELECT DATE_TRUNC('month', date_scanned) AS month -- truncating date_scanned to get month
    FROM receipt
    ORDER BY month DESC
    LIMIT 2 -- get recent two monthts
),
brand_counts AS ( -- CTE to calculate the counts of each brand for the recent months
    SELECT DATE_TRUNC('month', receipt.date_scanned) AS month, brand.name, COUNT(receipt.id) AS count -- extract month and get count of receipts
    FROM receipt
    JOIN receipt_item ON receipt.id = receipt_item.receipt_id -- join receipt_item with receipt with receipt_id
    JOIN brand ON receipt_item.brand_id = brand.id -- join receipt_item with brand with brand_id
    WHERE DATE_TRUNC('month', receipt.date_scanned) IN (SELECT month FROM recent_months) -- filter to include only those from the most recent two month found in the recent_month CTE
    GROUP BY month, brand.name
),
ranked_brands AS ( -- CTE to calculate the ranking of each brand for each month in brand_counts CTE
    SELECT month, name, count,
           RANK() OVER (PARTITION BY month ORDER BY count DESC) AS ranking
    FROM brand_counts
)
SELECT rb.month AS recent_month, rb.name AS brand_name, rb.ranking AS recent_ranking, COALESCE(rb_prev.ranking, 0) AS previous_ranking -- assign rank 0 to brands not present in previous month
FROM ranked_brands rb
LEFT JOIN ranked_brands rb_prev ON rb.month = rb_prev.month - INTERVAL '1 month' AND rb.name = rb_prev.name -- self join ranked_brands using month in rb and substracting 1 month in rb_prev 
WHERE rb.month = (SELECT month FROM recent_months LIMIT 1) -- select only the recent month as we are already self joining to map previous month
ORDER BY rb.count DESC
LIMIT 5; -- et top 5 results
```

## 3. Evaluate Data Quality Issues in the Data Provided

### 3.1. Missing values for columns that could potentially define relations

While a significant number of columns in `user` and `brand` tables had missing values, the columns that defined relations, i.e. `user_id` and `brand_id` did not have any missing values. Even for `receipt_id` in `receipt` table, there were no missing values. However, the problem was with the `rewardsReceiptItemList` in `receipt`, which I translated to the `receipt_item` table. None of the columns in this collection could form a relation to `brand` table. The only relevant column called `brandCode` had missing values and values that were not present in `brandCode`s of `brand` table.


In [23]:
# Checking for null values in brandCode column
brands["brandCode"].isnull().sum()

234

In [24]:
# Check if all brandCodes of receipt_items are present in brandCodes of brands
receipt_items["brandCode"].isin(brands["brandCode"]).all()

False

To form a concrete relation between the `receipt_item` and `brand` table, I had to include a new column in `receipt_item` called `brand_id` which references to `brand.id`. While this column does not contain any data as of now and would need some effort to be populated, but I feel this is the right way to fix the issue at hand.

There are other fields that have missing values but I think most of those qualify as field which may or may not have values based on the scenario.

Another data quality issue was with the `userId` values present in `receipts`. Not all `userId` were present in the `users` collection which could result in issues during `JOIN` operations.

In [25]:
# Check if all userId of receipts are present in id of users
receipts["userId"].isin(users["id"]).all()

False

### 3.2. Duplicate values for fields that need to be unique

In [26]:
# Fetch all users having the same id 5ff1e194b6a9d73a3a9f1052
users[users["id"] == "5ff1e194b6a9d73a3a9f1052"].head(5)

,id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI
5,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI


The `id` field of `users` contain duplicate values which should not have been the case. While loading data into the database, I decided to ignore the duplicate entries as all of their fields were same.

Same applies for the `barcode` and `brandCode` fields in `brands` which initially looked like unique fields but on analyzing I found that they contain duplicate values.

In [27]:
# Checking if barcode column is unique in raw data
brands["barcode"].nunique() == brands["barcode"].count()

False

In [28]:
# Checking if barcode column is unique in raw data
brands["brandCode"].nunique() == brands["brandCode"].count()

False

## 4. Communicate with Stakeholders

Hi Max,

I hope you are doing well. I am Srushti, an Analytics Engineer from the data team who has been working on the rewards analysis project for a couple of days now. I wanted your input on a few questions that I have. 

Executive project summary: Analyzing three primary datasets (users, receipts, and brands) to answer questions from the product team for next year's product roadmap. 

Questions about the data: 

1. Should the analysis include receipts that have missing user information? Please let me know which option you would like to go with:
    - We include all receipts and treat receipts with missing user information as guest receipts. This will help provide company-level rewards insights.
    - We exclude receipts with missing user information if we want to derive insights related to registered users.
    - Is there any other way of getting the exhaustive list of users from any other team where we can find missing user information?
2. Every receipt has multiple product items. While analyzing the receipts dataset, I observed that the brand codes in these product items are missing in the brand dataset. What approach would you like me to take? Here are a few options:
    - For missing brand codes in product items, I assume the brand is "Misc" and include them in the insights.
    - For product items with brand codes not present in the brand dataset, can we get brand information from any other team if we provide them with a list of brand codes?

Scaling and performance specifications:

1. Storing data on fast databases like PostgreSQL will become expensive as the business grows. Moving our application data to a data warehouse or cold storage like Amazon S3 will help reduce data storage costs. We can build an ETL pipeline to automate moving old data to a data warehouse and cold storage and set a retention policy for historical data. 
2. We should add testing infrastructure in data pipelines and CI/CD workflows to enforce data quality checks.
3. Data backups are going to be crucial in the production environment. For this, data replication will be required to ensure fault tolerance and data availability.
4. We need to ensure that data governance policies are set as per Legal team's guidelines and that only limited people in the company can access Personally Identifiable Information (PII) data.

Thank you very much. Please let me know if you have any questions for me.

Kind regards,

Srushti